# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [ ]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import matplotlib.pyplot as plt
import torch
# from torch.utils.data import DataLoader, TensorDataset
from pprint import pprint

from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing, ResNetProprioTunedRegression, ResNetProprioTunedSensorProcessing

from demonstration.demonstration import Demonstration
import sensorprocessing.sp_helper as sp_helper


Using device: cuda


### Exp-run initialization
Create the exp/run-s that describe the parameters of the training. 
Some of the code here is structured in such a way as to make the notebook automatizable with papermill.

In [ ]:
# *** Initialize the variables with default values 
# *** This cell should be tagged as parameters     
# *** If papermill is used, some of the values will be overwritten

# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
# run = "vgg19_128"
# run = "resnet50_128"
# run = "vgg19_256"
run = "resnet50_256"

# If it is set to true, the exprun will be recreated from scratch
creation_style = "exist-ok"
# If not None, set an external experiment path
external_path = None

***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\resnet50_256_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: sensorprocessing_propriotuned_cnn/resnet50_256 successfully loaded
Experiment:
    batch_size: 32
    data_dir: c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_propriotuned_cnn\resnet50_256
    epochs: 100
    exp_run_sys_indep_file: C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\sensorprocessing_propriotuned_cnn\resnet50_256.yaml
    experiment_name: sensorprocessing_propriotuned_cnn
    freeze_feature_extractor: true
  

In [ ]:
# create the necessary exp/run objects

if external_path:
    external_path = pathlib.Path(external_path)
    assert external_path.exists()
    Config().set_experiment_path(external_path)

exp = Config().get_experiment(experiment, run)
print(exp)

### Check the model for an encoding mode

Check whether we can load back the saved torch model. 

In [ ]:
# create the structure of the model with empty weights
if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp)
else:
    raise Exception(f"Unknown model {exp['model']}")

#if exp['loss'] == 'MSELoss':        
#    criterion = nn.MSELoss()
#elif exp['loss'] == 'L1Loss':
#    criterion = nn.L1Loss()

# load the model weights
modelfile = pathlib.Path(exp.data_dir(), 
                         exp["proprioception_mlp_model_file"])
assert modelfile.exists()
model.load_state_dict(torch.load(modelfile))

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\lboloni\AppData\Local\Temp\ipykernel_45352\721048493.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execu

<All keys matched successfully>

In [4]:
# Run the loaded torch model on all the specified testing data

transform = sp_helper.get_transform_to_sp(exp)

with torch.no_grad():
    for val in exp["testing_data"]:
        run, demo_name, camera = val
        exp_demo = Config().get_experiment("demonstration", run)
        demo = Demonstration(exp_demo, demo_name)
        for i in range(demo.metadata["maxsteps"]):
            sensor_readings, _ = demo.get_image(i, camera=camera, transform=transform, device=device)
            z = model.encode(sensor_readings)
            break

print(f"The shape of the output is: {z.shape}")
print(f"One example z looks like this:\n{z}")


***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\demonstration\_defaults_demonstration.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration\freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
The shape of the output is: torch.Size([1, 256])
One example z looks like this:
tensor([[ 1.3648,  0.6460, -4.5749, -0.3579, -2.7292, -0.3850,  0.0123, -1.3725,
         -1.6510,  1.9490,  5.1672,  2.7302,  1.8283, -0.6561,  1.8276, -4.4697,
          7.2294, -4.6204, -1.3750, -1.0661, -0.4525, -2.2820,  2.0185,  0.7959,
          0.9459, -0.5126,  3.0043,  0.5308,  0.3950,  0.0394,  0.0583, -4.1559,
          3.1655, -1.4671,  0.1335,  2.2880, -0.2642,  0.5997, -0.7335,  0.2674,
          3.6416, -0.5638, -0.1445,  2.0030,  5.493

### Create and use a SensorProcessing object

In [5]:
# Create the sensor processing object
sp = None
if exp["model"] == "VGG19ProprioTunedRegression":
    sp = VGG19ProprioTunedSensorProcessing(exp, device)
if exp["model"] == "ResNetProprioTunedRegression":
    sp = ResNetProprioTunedSensorProcessing(exp, device)
if sp is None:
    raise Exception(f"Could not instantiate sp for {exp['model']}")

c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\..\sensorprocessing\sp_propriotuned_cnn.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sel

In [6]:
# run all the testing images through the sensor processing
for val in exp["testing_data"]:
    run, demo_name, camera = val
    exp_demo = Config().get_experiment("demonstration", run)
    demo = Demonstration(exp_demo, demo_name)
    for i in range(demo.metadata["maxsteps"]):
        sensor_readings, _ = demo.get_image(i, camera=camera, transform=transform, device=device)
        z = sp.process(sensor_readings)
        break
# print the last z, as an example
print(z)
print(z.shape)

***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\demonstration\_defaults_demonstration.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\demonstration\freeform_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
[ 1.3647739   0.6460211  -4.5748835  -0.35791773 -2.7292213  -0.3849851
  0.01232137 -1.3725244  -1.6510304   1.9490317   5.1672044   2.7302008
  1.828321   -0.6560984   1.8276273  -4.469716    7.2294297  -4.6203585
 -1.374953   -1.0661334  -0.45254743 -2.2819533   2.0185444   0.7959301
  0.9459115  -0.51260585  3.0043335   0.5308151   0.3949905   0.03943964
  0.05829686 -4.1558743   3.1654809  -1.467056    0.13351682  2.2879868
 -0.26419     0.5997299  -0.7334907   0.2674005   3.64159    -0.5637553
 -0.14448145  2.0029674   5.493